In [1]:
import random
import pandas as pd

### Read files

In [2]:
df = pd.read_csv('data/Adventures-Table 1.csv')
df = df.dropna(subset=['Adventure'])
df['Base Rarity'] = df['Base Rarity'].astype('int')
df['Release'] = df['Release'].astype('int')
df.head()

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0


In [3]:
df2 = pd.read_csv('data/1 Stars-Table 1.csv')
df2['Availability'] = 'Permanent'
df2['Release'] = 1
df2.head()

,Adventure,Base Rarity,Class,Weapon,Element,Gender,Affiliation,Availability,Release
0,R.Warrior,1,Warrior,Sword,Ruby,Female,Egalus,Permanent,1
1,R.Knight,1,Knight,Lance,Ruby,Male,Fur Etopia,Permanent,1
2,R.Ranger,1,Ranger,Gun,Ruby,Male,Haiping,Permanent,1
3,R.Rogue,1,Rogue,Dagger,Ruby,Male,Fur Etopia,Permanent,1
4,R.Mage,1,Mage,Staff,Ruby,Female,Haiping,Permanent,1


In [4]:
df = df.append(df2)
df = df.reset_index(drop=True)
df = df.sort_values(by=['Release', 'Base Rarity'])
df.head()

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0


### Free Adventures

In [5]:
adv = df.loc[df['Release'] == 0]
adv['Adventure'].unique().tolist()

['Taiga', 'Zan', 'Rawn', 'Ryujin', 'Kaede', 'Sayori']

### Initial Rolls

In [6]:
x = 1

for i in range(50):
    x = 1 * 1.1**i
    print(i+71, x)

71 1.0
72 1.1
73 1.2100000000000002
74 1.3310000000000004
75 1.4641000000000004
76 1.6105100000000006
77 1.7715610000000008
78 1.9487171000000012
79 2.1435888100000016
80 2.357947691000002
81 2.5937424601000023
82 2.8531167061100025
83 3.138428376721003
84 3.452271214393104
85 3.7974983358324144
86 4.177248169415656
87 4.594972986357222
88 5.054470284992945
89 5.55991731349224
90 6.115909044841464
91 6.727499949325611
92 7.400249944258173
93 8.14027493868399
94 8.95430243255239
95 9.84973267580763
96 10.834705943388395
97 11.918176537727234
98 13.10999419149996
99 14.420993610649957
100 15.863092971714952
101 17.44940226888645
102 19.194342495775096
103 21.113776745352606
104 23.22515441988787
105 25.54766986187666
106 28.10243684806433
107 30.912680532870763
108 34.00394858615784
109 37.40434344477363
110 41.144777789250995
111 45.2592555681761
112 49.78518112499371
113 54.763699237493086
114 60.2400691612424
115 66.26407607736665
116 72.89048368510332
117 80.17953205361366
118 88.197

In [7]:
RATES = {1: 0.75, 2: 0.2, 3: 0.05}
FEATURED = {2: 0.04, 3: 0.01}
PITY_START = 71

RATES_GALA = {1: 0.72, 2: 0.2, 3: 0.08}
FEATURED_IMP = {2: 0, 3: 0.01}
FEATURED_TG = {2: 0, 3: 0.02}

def simulate_roll(num, release, rates=RATES, feature=FEATURED):
    result = pd.DataFrame()
    pity = 0
    for n in range(num):
        res, feat = roll(release, pity, rates, feature)
        if feat:
            pity = 0
        else:
            pity += 1
        result = result.append(res)
    return result

def roll(release, pity=0, rates=RATES, featured=FEATURED):
    pool = df.loc[df['Release'] <= release]
    die1 = random.random()
    # Check pity
    pity_rate = 0
    if pity > PITY_START:
        pity_rate = featured[3] * 1.1**(pity - PITY_START) - featured[3]
    # Check if get featured
    if die1 < featured[3] + pity_rate:
        return get_featured(3, release, pool), True
    # 1 STAR
    elif die1 <= rates[1] - pity_rate:
        return get_random_adventure(1, pool), False
    # 3 STARS
    elif die1 > 1 - (rates[3] - featured[3]):
        return get_random_adventure(3, pool), False
    # 2 STARS
    else:
        die2 = random.random()
        if die2 < featured[2] / rates[2]:
            return get_featured(2, release, pool), False
        else:
            return get_random_adventure(2, pool), False
            
def get_random_adventure(rarity, pool):
    return pool.loc[pool['Base Rarity'] == rarity].sample()

def get_featured(rarity, release, pool):
    row = pool.loc[(pool['Base Rarity'] == rarity) & (pool['Release'] == release)]
    if len(row) < 1:
        return None
    elif len(row) > 1:
        row = row.sample()
    return row

In [8]:
roll(1, 0)[0]

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
96,E.Cleric,1,Cleric,Wand,Emerald,Permanent,Female,Haiping,1


In [9]:
simulate_roll(30, 1)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
34,Vive,2,Rogue,Sword,Amber,Permanent,Female,Haiping,1
93,E.Ranger,1,Ranger,Dagger,Emerald,Permanent,Female,Haiping,1
99,T.Ranger,1,Ranger,Bow,Topaz,Permanent,Female,Dikaisia,1
14,Gifford,2,Mage,Staff,Sapphire,Permanent,Male,Fur Etopia,1
97,T.Warrior,1,Warrior,Axe,Topaz,Permanent,Female,Haiping,1
92,E.Knight,1,Knight,Axe,Emerald,Permanent,Male,Dikaisia,1
90,S.Cleric,1,Cleric,Staff,Sapphire,Permanent,Male,Haiping,1
112,A.Rogue,1,Rogue,Dagger,Amber,Permanent,Female,Haiping,1
85,S.Warrior,1,Warrior,Axe,Sapphire,Permanent,Female,Dikaisia,1
104,O.Knight,1,Knight,Mace,Onyx,Permanent,Male,Haiping,1


In [10]:
simulate_roll(100, 1)['Base Rarity'].value_counts()

1    75
2    17
3     8
Name: Base Rarity, dtype: int64

In [11]:
simulate_roll(10, 2)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
83,R.Mage,1,Mage,Staff,Ruby,Permanent,Female,Haiping,1
90,S.Cleric,1,Cleric,Staff,Sapphire,Permanent,Male,Haiping,1
6,Fritz,2,Rogue,Dagger,Ruby,Permanent,Male,Fur Etopia,1
24,Micah,2,Cleric,Mace,Topaz,Permanent,Male,Fur Etopia,1
108,O.Cleric,1,Cleric,Wand,Onyx,Permanent,Female,Dikaisia,1
90,S.Cleric,1,Cleric,Staff,Sapphire,Permanent,Male,Haiping,1
16,Kiris,2,Knight,Wand,Emerald,Permanent,Female,Dikaisia,1
80,R.Knight,1,Knight,Lance,Ruby,Permanent,Male,Fur Etopia,1
80,R.Knight,1,Knight,Lance,Ruby,Permanent,Male,Fur Etopia,1
98,T.Knight,1,Knight,Mace,Topaz,Permanent,Male,Egalus,1


In [12]:
simulate_roll(10, 3)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
38,Dred,2,Ranger,Lance,Onyx,Permanent,Male,Haiping,3
108,O.Cleric,1,Cleric,Wand,Onyx,Permanent,Female,Dikaisia,1
94,E.Rogue,1,Rogue,Lance,Emerald,Permanent,Male,Egalus,1
38,Dred,2,Ranger,Lance,Onyx,Permanent,Male,Haiping,3
111,A.Ranger,1,Ranger,Gun,Amber,Permanent,Female,Dikaisia,1
89,S.Mage,1,Mage,Sword,Sapphire,Permanent,Male,Fur Etopia,1
8,Senzo,2,Mage,Lance,Ruby,Permanent,Male,Dikaisia,1
79,R.Warrior,1,Warrior,Sword,Ruby,Permanent,Female,Egalus,1
85,S.Warrior,1,Warrior,Axe,Sapphire,Permanent,Female,Dikaisia,1
91,E.Warrior,1,Warrior,Gauntlets,Emerald,Permanent,Male,Fur Etopia,1


In [13]:
simulate_roll(10, 15, rates=RATES_GALA, featured=FEATURED_IMP)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
95,E.Mage,1,Mage,Sword,Emerald,Permanent,Female,Egalus,1
112,A.Rogue,1,Rogue,Dagger,Amber,Permanent,Female,Haiping,1
87,S.Ranger,1,Ranger,Dagger,Sapphire,Permanent,Female,Egalus,1
85,S.Warrior,1,Warrior,Axe,Sapphire,Permanent,Female,Dikaisia,1
80,R.Knight,1,Knight,Lance,Ruby,Permanent,Male,Fur Etopia,1
49,Erza,3,Mage,Staff,Ruby,Permanent,Female,Fur Etopia,8
87,S.Ranger,1,Ranger,Dagger,Sapphire,Permanent,Female,Egalus,1
112,A.Rogue,1,Rogue,Dagger,Amber,Permanent,Female,Haiping,1
80,R.Knight,1,Knight,Lance,Ruby,Permanent,Male,Fur Etopia,1
95,E.Mage,1,Mage,Sword,Emerald,Permanent,Female,Egalus,1


In [14]:
def get_roll_average(num_samples, num_rolls, release):
    stars2 = 0
    stars2f = 0
    stars3 = 0
    stars3f = 0
    for n in range(num_samples):
        r = simulate_roll(num_rolls, release)
        stars2 += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] != release)])
        stars2f += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] == release)])
        stars3 += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] != release)])
        stars3f += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] == release)])
    stars2 = round(stars2/num_samples, 2)
    stars2f = round(stars2f/num_samples, 2)
    stars3 = round(stars3/num_samples, 2)
    stars3f = round(stars3f/num_samples, 2)
    return stars2, stars2f, stars3, stars3f

In [15]:
get_roll_average(100, 100, 2)

(16.06, 4.8, 3.31, 2.04)

In [16]:
def roll_till_feature(num_samples, release):
    rolls = []
    for n in range(num_samples):
        r = 0
        while True:
            r += 1
            _, pity = roll(release, r)
            if pity:
                rolls.append(r)
                break
    return rolls

def roll_stop_at_feature(num, release, rates=RATES, feature=FEATURED):
    result = pd.DataFrame()
    pity = 0
    for n in range(num):
        res, feat = roll(release, pity, rates, feature)
        result = result.append(res)
        pity += 1
        if feat:
            return result, num - pity
    return result, 0

In [17]:
roll_till_feature(10, 2)

[97, 78, 9, 96, 5, 95, 33, 46, 95, 92]

In [18]:
rolls = roll_till_feature(100, 2)
sum(rolls) / len(rolls)

64.23

In [19]:
max(rolls), min(rolls)

(110, 2)

### Team

In [20]:
team = adv.copy()
# 50 initial free rolls
roll_res = simulate_roll(50, 1)
team = team.append(roll_res)
# 30 free rolls for 1st week
rolls_available = 30
print("Rolled: {}\nRolls Available: {}".format(50, rolls_available))
roll_res.loc[roll_res['Base Rarity'] > 1]

Rolled: 50
Rolls Available: 30


,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
21,Ozanne,2,Warrior,Axe,Topaz,Permanent,Female,Dikaisia,1
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0
32,Phillip,2,Knight,Lance,Amber,Permanent,Male,Fur Etopia,1
23,Rea,2,Ranger,Bow,Topaz,Permanent,Female,Egalus,1
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0
12,Esable,2,Knight,Bow,Sapphire,Permanent,Female,Haiping,1
23,Rea,2,Ranger,Bow,Topaz,Permanent,Female,Egalus,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
28,Jun,2,Mage,Staff,Onyx,Permanent,Male,Haiping,1


In [21]:
# Week 6
featured = 'Tamamo'
week = 6
rolls_available += 50
to_roll = 40
roll_res, rolls_left = roll_stop_at_feature(to_roll, week)
team = team.append(roll_res)
rolls_available -= (to_roll - rolls_left)
print("Rolled: {}\nRolls Available: {}".format(to_roll-rolls_left, rolls_available))
roll_res.loc[roll_res['Base Rarity'] > 1]

Rolled: 40
Rolls Available: 40


,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
13,Offenbach,2,Ranger,Dagger,Sapphire,Permanent,Male,Egalus,1
21,Ozanne,2,Warrior,Axe,Topaz,Permanent,Female,Dikaisia,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,Haiping,1
8,Senzo,2,Mage,Lance,Ruby,Permanent,Male,Dikaisia,1
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0


In [22]:
# Week 8
featured = 'Erza'
week = 8
rolls_available += 20
to_roll = 40
roll_res, rolls_left = roll_stop_at_feature(to_roll, week)
team = team.append(roll_res)
rolls_available -= (to_roll - rolls_left)
print("Rolled: {}\nRolls Available: {}".format(to_roll-rolls_left, rolls_available))
roll_res.loc[roll_res['Base Rarity'] > 1]

Rolled: 40
Rolls Available: 20


,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
48,Leyin,2,Cleric,Gun,Topaz,Permanent,Female,Egalus,8
48,Leyin,2,Cleric,Gun,Topaz,Permanent,Female,Egalus,8
9,Panlei,2,Cleric,Wand,Ruby,Permanent,Female,Haiping,1
36,Elsie,2,Mage,Sword,Topaz,Permanent,Female,Fur Etopia,2
48,Leyin,2,Cleric,Gun,Topaz,Permanent,Female,Egalus,8
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
20,Canis,3,Warrior,Gauntlets,Emerald,Permanent,Male,Egalus,1
48,Leyin,2,Cleric,Gun,Topaz,Permanent,Female,Egalus,8
30,Nevermore,3,Ranger,Bow,Onyx,Permanent,Male,Fur Etopia,1
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0


In [23]:
# Week 10
featured = 'Syllia'
week = 10
rolls_available += 20
to_roll = rolls_available
roll_res, rolls_left = roll_stop_at_feature(to_roll, week, RATES_GALA, FEATURED_IMP)
team = team.append(roll_res)
rolls_available -= (to_roll - rolls_left)
print("Rolled: {}\nRolls Available: {}".format(to_roll-rolls_left, rolls_available))
roll_res.loc[roll_res['Base Rarity'] > 1]

Rolled: 29
Rolls Available: 11


,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1
44,Glace,2,Rogue,Dagger,Sapphire,Permanent,Male,Egalus,6
12,Esable,2,Knight,Bow,Sapphire,Permanent,Female,Haiping,1
30,Nevermore,3,Ranger,Bow,Onyx,Permanent,Male,Fur Etopia,1
33,Yawen,2,Ranger,Dagger,Amber,Permanent,Female,Haiping,1
33,Yawen,2,Ranger,Dagger,Amber,Permanent,Female,Haiping,1
30,Nevermore,3,Ranger,Bow,Onyx,Permanent,Male,Fur Etopia,1
49,Erza,3,Mage,Staff,Ruby,Permanent,Female,Fur Etopia,8
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0
9,Panlei,2,Cleric,Wand,Ruby,Permanent,Female,Haiping,1


In [24]:
team['Count'] = team.groupby('Adventure')['Release'].transform('count')
team = team.drop_duplicates()
team.loc[team['Base Rarity'] >= 3]

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1,2
20,Canis,3,Warrior,Gauntlets,Emerald,Permanent,Male,Egalus,1,1
30,Nevermore,3,Ranger,Bow,Onyx,Permanent,Male,Fur Etopia,1,3
49,Erza,3,Mage,Staff,Ruby,Permanent,Female,Fur Etopia,8,1
52,Syllia,3,Ranger,Bow,Emerald,Imperial,Female,None,10,1


In [25]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Ruby')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
49,Erza,3,Mage,Staff,Ruby,Permanent,Female,Fur Etopia,8,1
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0,3
8,Senzo,2,Mage,Lance,Ruby,Permanent,Male,Dikaisia,1,1
9,Panlei,2,Cleric,Wand,Ruby,Permanent,Female,Haiping,1,2


In [26]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Sapphire')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1,1
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0,1
12,Esable,2,Knight,Bow,Sapphire,Permanent,Female,Haiping,1,2
13,Offenbach,2,Ranger,Dagger,Sapphire,Permanent,Male,Egalus,1,2
44,Glace,2,Rogue,Dagger,Sapphire,Permanent,Male,Egalus,6,1


In [27]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Emerald')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
20,Canis,3,Warrior,Gauntlets,Emerald,Permanent,Male,Egalus,1,1
52,Syllia,3,Ranger,Bow,Emerald,Imperial,Female,None,10,1
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0,1
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1,1
18,Vind,2,Mage,Sword,Emerald,Permanent,Male,Egalus,1,1


In [28]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Topaz')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0,4
21,Ozanne,2,Warrior,Axe,Topaz,Permanent,Female,Dikaisia,1,2
23,Rea,2,Ranger,Bow,Topaz,Permanent,Female,Egalus,1,2
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,Haiping,1,1
48,Leyin,2,Cleric,Gun,Topaz,Permanent,Female,Egalus,8,4
36,Elsie,2,Mage,Sword,Topaz,Permanent,Female,Fur Etopia,2,1


In [29]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Onyx')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
30,Nevermore,3,Ranger,Bow,Onyx,Permanent,Male,Fur Etopia,1,3
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0,3
28,Jun,2,Mage,Staff,Onyx,Permanent,Male,Haiping,1,1
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1,1


In [30]:
team.loc[(team['Base Rarity'] > 1) & (team['Element'] == 'Amber')].sort_values(by='Base Rarity', ascending=False)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1,2
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0,2
32,Phillip,2,Knight,Lance,Amber,Permanent,Male,Fur Etopia,1,1
33,Yawen,2,Ranger,Dagger,Amber,Permanent,Female,Haiping,1,2
